We have to create few temporary tables to produce the final reports/stats
as listed in the https://www.iplt20.com/stats/2008 we will try for atleast 95% accuracy on this reports as this data is collected from web not from offical IPL website.

list of temporary view/table created:
1) BATTER_INNS_PLAYED: stages infomation about batsmen's(total_matches_played ,innings_played, not_out innings per each ipl season)

2) TOTAL_RUNS_AND_BALLS_FACED: season runs scored and balls faced

3) batter_season_highg_scores: batsmen's seasons higest score

4) centuries: bastsmen's season centuries 

5) boundry_count: total batsmen's 4's and 6's for season.

6) fastest_fifty_hundred: used to get fastest fifty and hundred runs

This will temporary tables will help us create necessary reports

There are 13 reports which are related to batsmen's categorised at season level and match level.

------------------ Season level stats ----------------
1) Orange Cap
2) RuPay On-The-Go 4s of the Season
3) Angel One Super Sixes of the Season
4) Most Fifties
5) Most Centuries
6) Highest Scores
7) Punch.ev Electric Striker of the Season
8) Best Batting Averages

------------------ Innings level stats ---------------
1) Most fours 
2) Most sixes
3) Fastest Fifty
4) Fastest Centuries
5) Best Batting Strike Rate --- will not do, due to data inaccuracy

In [0]:
client_id = dbutils.secrets.get(scope="key-vault-secret", key="sp-client-id")
tenant_id = dbutils.secrets.get(scope="key-vault-secret", key="sp-tenent-id")
client_secret_key = dbutils.secrets.get(scope="key-vault-secret", key="sp-secret-key")

spark.conf.set("fs.azure.account.auth.type.iplsandbox.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.iplsandbox.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.iplsandbox.dfs.core.windows.net", f"{client_id}")
spark.conf.set("fs.azure.account.oauth2.client.secret.iplsandbox.dfs.core.windows.net", f"{client_secret_key}")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.iplsandbox.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

In [0]:
%sql
---Databricks catalouge databge ipl_battting_stats
CREATE DATABASE IF NOT EXISTS IPL_BATTING_STATS;
USE IPL_BATTING_STATS;

Generate Season level stats

In [0]:
BATTER_INNS_PLAYED="""select 
season_year,
player_id,
player_team,
count(distinct match_id) as total_matches_played,
sum(inn) as innings_played,
(sum(inn) -sum(out_ind) )as not_out
from 
(
select a.match_id,
	   a.player_id,
	   a.season_year,
	   a.player_team_abbrivation as player_team,
       case when (a.player_id=b.striker or a.player_id=b.non_striker) then 1 else 0 end as inn,
       case when a.player_id=c.player_out then 1 else 0 end as out_ind 
from ipl_data.player_match a
left join 
ipl_data.ball_by_ball_stats b
on a.match_id=b.match_id and a.season_year=b.season and (a.player_id=b.striker or a.player_id=b.non_striker)
left join 
ipl_data.ball_by_ball_stats c on c.match_id=a.match_id and a.player_id=c.player_out and c.player_out is not null
GROUP BY 1,2,3,4,5,6) a
group by 1,2,3""" 

total_runs_and_balls_faced="""select 
  season, 
  striker, 
  sum(runs_scored) as scored_match_runs, 
  sum(
    case when extra_type in (
      'No Extras', 'Byes', 'Legbyes', 'legbyes', 'byes', 'Noballs', 'noballs'
    ) then 1 else 0 end
  ) as balls_faced 
from 
  ipl_data.ball_by_ball_stats 
group by 1, 2;"""

batters_season_highg_scores="""
select 
season,
striker,
runs
from
(
select 
season,
striker,
match_id,
sum(runs_scored) as runs,
row_number() over( partition by  season,striker order by season asc , sum(runs_scored) desc) as runs_rank
from ipl_data.ball_by_ball_stats
group by 1,2,3
) a where  runs_rank=1;"""

centuries="""
select 
season,
striker, 
sum(case when runs>=50 and runs<100 then 1 else 0 end) as fifties ,
sum(case when runs>=100 then 1 else 0 end) as hundreds 
from (
select 
season,
striker,
match_id,
sum(runs_scored) as runs
from ipl_data.ball_by_ball_stats
group by 1,2,3 ) a 
group by season,striker;"""

boundry_count="""
select 
season,
striker,
sum (case when runs_scored=4 then 1 else 0 end ) as fours,
sum (case when runs_scored=6 then 1 else 0 end ) as sixes
from ipl_data.ball_by_ball_stats
where runs_scored in (4,6) 
group by 1,2  ; """

fastest_fifty_hundred="""
select 
match_id,
season,
striker,
sum(runs_scored) over(partition by match_id,season,striker order by over_id asc, ball_id asc) as run_mark,
row_number() over(partition by match_id,season,striker order by match_id ) as balls_faced_to_get_runs
from ipl_data.ball_by_ball_stats 
where  extra_type in ('No Extras','Byes','Legbyes','legbyes','byes')"""

fours_sixes="""
select 
a.match_id,
a.season,
a.striker,
a.team_batting, --battingteam_sk
a.team_bowling, --bowlingteam_sk
sum(a.runs_scored) as runs_scored,
sum (case when runs_scored=4 then 1 else 0 end ) as fours,
sum (case when runs_scored=6 then 1 else 0 end ) as sixes
from ipl_data.ball_by_ball_stats  a
group by 1,2,3,4,5"""

most_fours_sixes="""
select 
a.season as season_year,
p.player_name as PLAYER,
a.runs_scored as RUNS,
a.player_team,
a.BF as BF,
coalesce(ROUND((cast(NULLIF(a.runs_scored,0) as DECIMAL(10,2))/cast(NULLIF(a.BF,0) as decimal(10,2))*100),2),0) as SR,
a.fours as `4S`,
a.sixes as `6S`,
a.AGAINST,
m.venue_name as VENU,
m.match_date as MATCH_DATE
from
(select 
match_id,
season,
striker,
b.team_name_abbrivation as player_team,
c.team_name_abbrivation as AGAINST ,
sum(runs_scored) as runs_scored,
sum(case when extra_type in ('No Extras','Byes','Legbyes','legbyes','byes') then 1 else 0 end ) as BF,
sum (case when runs_scored=4 then 1 else 0 end ) as fours,
sum (case when runs_scored=6 then 1 else 0 end ) as sixes
from ipl_data.ball_by_ball_stats a
left join 
ipl_data.teams b on a.team_batting=b.team_id
 left join 
ipl_data.teams c on a.team_bowling=c.team_id
group by 1,2,3,4,5
) a
left join 
ipl_data.matches m on a.match_id=m.match_id
left join 
ipl_data.players_data p on p.player_id=a.striker"""


spark.sql(BATTER_INNS_PLAYED).createOrReplaceTempView('BATTER_INNS_PLAYED')

spark.sql(total_runs_and_balls_faced).createOrReplaceTempView('TOTAL_RUNS_AND_BALLS_FACED')

spark.sql(batters_season_highg_scores).createOrReplaceTempView('BATTERS_SEASON_HIGH_SCORES')

spark.sql(centuries).createOrReplaceTempView('CENTURIES')

spark.sql(boundry_count).createOrReplaceTempView('BOUNDRY_COUNT')

spark.sql(fastest_fifty_hundred).createOrReplaceTempView('fastest_fifty_hundred')

spark.sql(fours_sixes).createOrReplaceTempView('fours_sixes')

spark.sql(most_fours_sixes).createOrReplaceTempView('most_fours_sixes')

In [0]:
%sql
select * from fours_sixes

match_id,season,striker,team_batting,team_bowling,runs_scored,fours,sixes
598037,2013,340,11,5,4,0,0
598010,2013,85,5,1,36,3,1
501219,2011,169,5,1,11,1,0
501206,2011,208,7,6,14,2,0
980964,2016,110,2,1,4,0,0
336030,2008,95,1,3,27,1,2
335992,2008,26,4,5,20,3,1
336039,2008,113,2,8,3,0,0
336002,2008,3,1,7,19,1,0
829794,2015,20,3,7,39,2,2


In [0]:
%sql
CREATE EXTERNAL LOCATION IF NOT EXISTS my_external_gold_location
URL 'abfss://ipldata@iplsandbox.dfs.core.windows.net/gold/'
WITH (STORAGE CREDENTIAL adls_data_access);

In [0]:
gold_path = dbutils.widgets.get("dbfs_gold_path")

ORANGE_CAP="""select 
a.season_year,
a.player_id as PLAYER, 
h.player_name,
a.player_team,
coalesce(b.scored_match_runs,0) as RUNS,
coalesce(a.total_matches_played,0) as MAT ,
coalesce(a.innings_played,0) as INNS,
coalesce(a.not_out,0) as NO,
coalesce(c.runs,0) as HS,
coalesce(ROUND(cast(NULLIF(b.scored_match_runs,0) as DECIMAL(10,2))/cast(NULLIF((NULLIF(a.innings_played,0) -NULLIF(a.not_out,0)) ,0) as decimal(10,2)),2),0) as AVG,
coalesce(b.balls_faced,0) as BF,
coalesce(ROUND((cast(NULLIF(b.scored_match_runs,0) as DECIMAL(10,2))/cast(NULLIF(b.balls_faced,0) as decimal(10,2))*100),2),0) as SR,
coalesce(d.hundreds,0) as `100`,
coalesce(d.fifties,0) as `50`,
coalesce(f.fours,0) as `4S`,
coalesce(f.sixes,0) as `6S`
from BATTER_INNS_PLAYED a 
left join 
total_runs_and_balls_faced b on a.season_year=b.season and a.player_id=b.striker
left join
BATTERS_SEASON_HIGH_SCORES c on a.season_year=c.season and a.player_id=c.striker
left join 
centuries d on a.season_year=d.season and a.player_id=d.striker
left join
boundry_count f on a.season_year=f.season and a.player_id=f.striker
left join 
ipl_data.players_data h on a.player_id=h.player_id
order by a.season_year asc,RUNS desc;"""


spark.sql(f"""
    CREATE OR REPLACE TABLE orange_cap
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/batting_stats/season/orange_cap"
    AS {ORANGE_CAP}
""")


RuPay_On_The_Go_4s_of_the_Season="""select 
season_year,
Player,
player_name,
player_team,
"4S",
mat,
inns,
no,
runs,
hs,
"AVG",
bf,
sr,
`100`,
`50`,
`6S`
from ORANGE_CAP order by season_year asc, `4S` desc"""

spark.sql(f"""
    CREATE OR REPLACE TABLE RuPay_On_The_Go_4s_of_the_Season
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/batting_stats/season/RuPay_On_The_Go_4s_of_the_Season"
    AS {RuPay_On_The_Go_4s_of_the_Season}
""")

Angel_One_Super_Sixes_of_the_Season="""select 
season_year,
Player,
player_name,
player_team,
"6S",
mat,
inns,
no,
runs,
hs,
"AVG",
bf,
sr,
`100`,
`50`,
`4S`
from ORANGE_CAP order by season_year asc, `6S` desc"""

spark.sql(f"""
    CREATE OR REPLACE TABLE Angel_One_Super_Sixes_of_the_Season
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/batting_stats/season/Angel_One_Super_Sixes_of_the_Season"
    AS {Angel_One_Super_Sixes_of_the_Season}
""")

Most_Fifties="""select 
season_year,
Player,
player_name,
player_team,
`50`,
mat,
inns,
no,
runs,
hs,
AVG,
bf,
sr,
`100`,
`4S`,
`6S`
from ORANGE_CAP order by season_year asc, `50` desc"""

spark.sql(f"""
    CREATE OR REPLACE TABLE Most_Fifties
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/batting_stats/season/Most_Fifties"
    AS {Most_Fifties}
""")

Most_Centuries="""select 
season_year,
Player,
player_name,
player_team,
`100`,
mat,
inns,
no,
runs,
hs,
AVG,
bf,
sr,
`50`,
`4S`,
`6S`
from ORANGE_CAP order by season_year asc, `100` desc """


spark.sql(f"""
    CREATE OR REPLACE TABLE Most_Centuries
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/batting_stats/season/Most_Centuries"
    AS {Most_Centuries}
""")

Highest_Scores="""select 
season_year,
Player,
player_name,
player_team,
hs,
mat,
inns,
no,
runs,
AVG,
bf,
sr,
`100`,
`50`,
`4S`,
`6S`
from ORANGE_CAP order by season_year asc, hs desc """

spark.sql(f"""
    CREATE OR REPLACE TABLE Highest_Scores
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/batting_stats/season/Highest_Scores"
    AS {Highest_Scores}
""")

Punch_ev_Electric_Striker_of_the_Season="""select 
season_year,
Player,
player_name,
player_team,
sr,
mat,
inns,
no,
runs,
hs,
AVG,
bf,
`100`,
`50`,
`4S`,
`6S`
from ORANGE_CAP order by season_year asc, sr desc """

spark.sql(f"""
    CREATE OR REPLACE TABLE Punch_ev_Electric_Striker_of_the_Season
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/batting_stats/season/Punch_ev_Electric_Striker_of_the_Season"
    AS {Punch_ev_Electric_Striker_of_the_Season}
""")

Best_Batting_Averages="""select 
season_year,
Player,
player_name,
player_team,
AVG,
mat,
inns,
no,
runs,
hs,
bf,
sr,
`100`,
`50`,
`4S`,
`6S`
from ORANGE_CAP order by season_year asc, "AVG" desc """

spark.sql(f"""
    CREATE OR REPLACE TABLE Best_Batting_Averages
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/batting_stats/season/Best_Batting_Averages"
    AS {Best_Batting_Averages}
""")

Out[7]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

Innings Stats

In [0]:
most_fours="""select 
season_year,
player
player_team,
`4S`,
RUNS,
BF,
SR,
`6S`,
AGAINST,
VENU,
MATCH_DATE
from most_fours_sixes order by season_year asc, `4S` desc;"""

spark.sql(f"""
    CREATE OR REPLACE TABLE most_fours
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/batting_stats/innings/most_fours"
    AS {most_fours}
""")


most_sixes="""select 
season_year,
player
player_team,
`6S`,
RUNS,
BF,
SR,
`4S`,
AGAINST,
VENU,
MATCH_DATE
from most_fours_sixes order by season_year asc, `6S` desc;"""

spark.sql(f"""
    CREATE OR REPLACE TABLE most_sixes
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/batting_stats/innings/most_sixes"
    AS {most_sixes}
""")


#------------------------------------------ Fastest Fifties ------------------------------------------





fastest_fifties=""" 
select a.season as season_year,
           a.player_name as PLAYER,
	   b.player_team,
	   b.runs_scored as RUNS, 
	   a.balls_faced_to_get_to_fifty_runs as BF,
	   Round((cast(b.runs_scored as DECIMAL(10,2))/cast(a.balls_faced_to_get_to_fifty_runs as DECIMAL(10,2))*100),2) as SR,
           b.AGAINST,
	   b.venue,
	   b.MATCH_DATE	   
from
(select 
match_id,
season,
striker,
p.player_name,
min(balls_faced_to_get_runs) as balls_faced_to_get_to_fifty_runs
from 
fastest_fifty_hundred a 
left join
ipl_data.players_data p on a.striker=p.player_id
 where a.run_mark >=50 and a.run_mark <60
group by 1,2,3,4 ) a
left join 
(select 
a.match_id,
a.season,
a.striker,
b.team_name_abbrivation as player_team,
c.team_name_abbrivation as AGAINST ,
m.venue_name as VENUE,
m.match_date as MATCH_DATE,
a.runs_scored
from fours_sixes a
 left join
ipl_data.teams b on a.team_batting=b.team_id
 left join 
ipl_data.teams c on a.team_bowling=c.team_id
 left join 
ipl_data.matches m on a.match_id=m.match_id) b on a.match_id=b.match_id and a.season=b.season and a.striker=b.striker
order by a.season asc, bf asc;"""


spark.sql(f"""
    CREATE OR REPLACE TABLE fastest_fifties
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/batting_stats/innings/fastest_fifties"
    AS {fastest_fifties}
""")

#------------------------------------------ Fastest Centuries ------------------------------------------
Fastest_Centuries="""
select     a.season as season_year,
           a.player_name as PLAYER,
	   b.player_team,
	   b.runs_scored as RUNS, 
	   a.balls_faced_to_get_to_hundred_runs as BF,
	   Round((cast(b.runs_scored as DECIMAL(10,2))/cast(a.balls_faced_to_get_to_hundred_runs as DECIMAL(10,2))*100),2) as SR,
           b.AGAINST,
	   b.venue,
	   b.MATCH_DATE
	   
from
(select 
match_id,
season,
striker,
p.player_name,
min(balls_faced_to_get_runs) as balls_faced_to_get_to_hundred_runs
from 
fastest_fifty_hundred a 
left join
ipl_data.players_data p on a.striker=p.player_id
 where a.run_mark >=100
group by 1,2,3,4 ) a
left join 
	(select 
	a.match_id,
	a.season,
	a.striker,
	b.team_name_abbrivation as player_team,
	c.team_name_abbrivation as AGAINST ,
	m.venue_name as VENUE,
	m.match_date as MATCH_DATE,
	a.runs_scored
	from fours_sixes a
left join
	ipl_data.teams b on a.team_batting=b.team_id
left join 
	ipl_data.teams c on a.team_bowling=c.team_id
left join 
	ipl_data.matches m on a.match_id=m.match_id) b on a.match_id=b.match_id and a.season=b.season and a.striker=b.striker
order by a.season asc, bf asc;"""


spark.sql(f"""
    CREATE OR REPLACE TABLE Fastest_Centuries
    USING DELTA
    PARTITIONED BY (season_year)
    LOCATION "{gold_path}/batting_stats/innings/Fastest_Centuries"
    AS {Fastest_Centuries}
""")

Out[8]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]